# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras import models,layers
from keras import regularizers


In [2]:
df = pd.read_csv('./creditcard.csv')
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)
# data["Class"].value_counts()
# x_data= data.drop(columns="Class", axis=0)
# label = data["Class"]

In [27]:
# min_d = data.min()
# max_d = data.max()
# normalized_df=(data - min_d)/(max_d - min_d)
# normalized_df.head()
data.sample()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1329,145675.0,1.470336,-0.822352,-0.606526,1.376553,-0.410095,0.044315,-0.062853,-0.058148,0.645821,...,0.033067,-0.379533,0.167913,0.63473,-0.443574,-0.990643,0.011982,0.02866,260.0,0


In [31]:
train_data = data.sample(frac= 0.7, random_state=125)
test_data =  data.drop(train_data.index)
# train_data.index.

In [11]:
train_labels = train_data.iloc[:,-1]
train_data = train_data.iloc[:,0:-1]
test_labels = test_data.iloc[:,-1]
test_data = test_data.iloc[:,0:-1]

In [12]:
train_mean = train_data.mean(axis=0) # taking the mean of 
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [13]:
train_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,...,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03
mean,-2.529826e-17,1.504199e-16,3.512752e-17,-1.763728e-16,-2.280066e-16,-3.212235e-16,2.466580e-16,-1.944904e-16,2.500620e-17,3.055128e-16,...,4.045506e-17,-4.535962e-17,-3.867250e-18,2.195470e-18,-4.185492e-17,9.249173e-17,7.170526e-18,-5.103964e-17,7.833195e-17,3.176785e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.916265e+00,-6.488845e+00,-9.058283e+00,-5.851657e+00,-2.162705e+00,-6.380448e+00,-4.469182e+00,-9.091446e+00,-1.119501e+01,-6.483506e+00,...,-1.718225e+01,-1.068850e+01,-9.006882e+00,-1.940651e+01,-3.461272e+00,-7.858383e+00,-3.227427e+00,-8.891680e+00,-1.953299e+01,-3.868215e-01
25%,-8.560763e-01,-6.346140e-02,-4.215537e-01,-5.334623e-02,-6.395648e-01,-7.304335e-02,-4.393857e-01,6.127158e-02,-8.841862e-02,-3.082447e-01,...,-2.643182e-01,-1.778484e-01,-5.278637e-01,-1.834537e-01,-6.009934e-01,-5.327550e-01,-6.817694e-01,-1.291275e-01,-1.439579e-01,-3.747286e-01
50%,-1.704201e-01,1.898064e-01,-1.796547e-01,2.873250e-01,-2.812188e-01,1.875616e-01,-3.704668e-02,2.765086e-01,-1.006972e-02,1.990615e-01,...,-9.618695e-02,-7.133366e-02,-1.592891e-02,-1.527709e-02,7.373741e-02,2.307495e-02,-3.673616e-02,-3.024762e-02,2.353205e-02,-3.070000e-01
75%,9.718638e-01,5.435632e-01,1.503838e-01,5.177168e-01,3.465558e-01,4.131710e-01,3.551652e-01,4.123250e-01,1.166541e-01,5.565553e-01,...,1.584344e-01,8.190104e-02,5.446272e-01,1.841757e-01,7.788549e-01,6.052032e-01,5.719055e-01,2.305977e-01,3.027582e-01,-3.468883e-02
max,1.668139e+00,7.928521e-01,6.626973e+00,1.114858e+00,3.981277e+00,3.558234e+00,4.207138e+00,1.653910e+00,5.411464e+00,3.470843e+00,...,1.001518e+01,1.259680e+01,8.488508e+00,8.156511e+00,1.890908e+00,4.417233e+00,3.122653e+00,6.604315e+00,4.072320e+00,1.530809e+01


In [14]:
# train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [15]:
train_data = train_data.to_numpy()
train_labels = train_labels.to_numpy().astype('float64')

test_data = test_data.to_numpy()
test_labels = test_labels.to_numpy().astype('float64')

In [16]:
model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [17]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [18]:
model.fit(train_data, train_labels, epochs=50,validation_split = 0.3)

Epoch 1/50
31/31 [==============================] - 2s 26ms/step - loss: 0.9450 - accuracy: 0.4712 - val_loss: 0.6887 - val_accuracy: 0.6643
Epoch 2/50
31/31 [==============================] - 0s 3ms/step - loss: 0.8352 - accuracy: 0.6010 - val_loss: 0.6684 - val_accuracy: 0.8575
Epoch 3/50
31/31 [==============================] - 0s 4ms/step - loss: 0.7410 - accuracy: 0.6971 - val_loss: 0.6562 - val_accuracy: 0.9106
Epoch 4/50
31/31 [==============================] - 0s 3ms/step - loss: 0.6822 - accuracy: 0.7967 - val_loss: 0.6447 - val_accuracy: 0.9082
Epoch 5/50
31/31 [==============================] - 0s 4ms/step - loss: 0.6605 - accuracy: 0.8288 - val_loss: 0.6313 - val_accuracy: 0.9010
Epoch 6/50
31/31 [==============================] - 0s 3ms/step - loss: 0.6512 - accuracy: 0.8311 - val_loss: 0.6152 - val_accuracy: 0.9010
Epoch 7/50
31/31 [==============================] - 0s 4ms/step - loss: 0.6345 - accuracy: 0.8507 - val_loss: 0.5884 - val_accuracy: 0.9130
Epoch 8/50
31/31 [=

In [19]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 2ms/step - loss: 0.3757 - accuracy: 0.9458


In [20]:
loss

0.3757486641407013

In [21]:
accuracy

0.9457626938819885

In [22]:
predictions = model.predict(test_data)

In [23]:
y_pred = (predictions > 0.5)

In [24]:
con = tf.math.confusion_matrix(
    test_labels, y_pred, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None
)

In [25]:
con

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[442,   6],
       [ 26, 116]])>

In [26]:
result = accuracy * 100
result

94.57626938819885